### IMPORT LIBRARY

In [22]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [23]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [24]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [25]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2021
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2018
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2019
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2016
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2020
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2017


In [26]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2021/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2018/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2019/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q3
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q4
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q2
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2022/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2016/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2020/Q4
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q2
/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q1
Folder name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2017/Q4


In [27]:
quarter = ['Q1']#Chọn năm cần import
year = ['2023']#Chọn quý cần import
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub1:
    if i.split('/')[-2] in year and i.split('/')[-1] in quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q1/FDI_20230331.xlsx


In [28]:
url = []
for i in list_file:
    data_key = i.split('/')[-1].split('.')[0].split('_')[-1]
    if str(data_key) not in ('x'):
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/FDI/Flat file/2023/Q1/FDI_20230331.xlsx']

### IMPORT DATA

In [29]:
'''Prepare ingredients'''
columns_that_need_unidecode=['Project_Name', 'Business_Name', 'Investor_Name', 'Investor_Nationality'
                             , 'Industry_Lv1', 'Investment_Form', 'City', 'District'
                            ]
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat_fdi = pd.DataFrame()
df_flat_fdi = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file
    #-------------------------------------------------------
    '''Validation step'''
    #Remove unfortmated values
    #data = remove_unformated_character(type_sector = 2, sector = sector, file_name = file_name, data = data)
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City', 'District', 'Project_Name', 'Investor_Nationality', 'Industry_Lv1', 'Investment_Form']
    lst_cls = ['City', 'District', 'Project_Name', 'Investor_Nationality', 'Industry_Lv1', 'Investment_Form']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    print(df_dict)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
        df_temp_flat_fdi = pd.concat([df_temp_flat_fdi, data], axis=0)
        df_flat_fdi = tracking_flat_file(df_temp_flat_fdi, sector)
        # insert_to_fresh(file_url, data, engine)
    else:
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

<function check_dictionary.<locals>.<lambda> at 0x0000023B95273C70>
<function check_dictionary.<locals>.<lambda> at 0x0000023B95273C70>
<function check_dictionary.<locals>.<lambda> at 0x0000023B94DB72E0>
<function check_dictionary.<locals>.<lambda> at 0x0000023B94DB7D90>
<function check_dictionary.<locals>.<lambda> at 0x0000023B94DB7910>
<function check_dictionary.<locals>.<lambda> at 0x0000023B94DB72E0>


100%|██████████| 1/1 [00:38<00:00, 38.28s/it]

        File_Name                                     Missing_Values  \
0    FDI_20230331                                               None   
0    FDI_20230331                   du an top speed fashion viet nam   
1    FDI_20230331                               du an intre viet nam   
2    FDI_20230331  du an thanh lap cong ty tnhh mot thanh vien go...   
3    FDI_20230331      cong ty co phan dich vu bao ve secom viet nam   
4    FDI_20230331                    cong ty tnhh dc vietnam trading   
5    FDI_20230331                        cong ty tnhh adminuniversal   
6    FDI_20230331                      du an san xuat tu bep highend   
7    FDI_20230331                          san xuat soi khanh nghiep   
8    FDI_20230331                                              l-bit   
9    FDI_20230331                              cong ty tnhh utt vina   
10   FDI_20230331                     nha may gia cong tranit vina 2   
11   FDI_20230331            cong ty tnhh cong nghe ruixing viet

### SEND EMAIL

In [30]:
list_df_flat = [df_flat_fdi]
if len(df_dict) != 0:
    print(colored('Missing values in dictionary','yellow'))
    df_noti_html = convert_df_to_html(type_html = 2, df = df_dict, cnxn = engine)
    to_email = ['nngocphuonguyen@savills.com.vn']
    run_email(email_type = 2, user_email = to_email, df_noti_html = df_noti_html)
else:
    print('Imported data succesfully')
    df_flat_html, df_query_html = convert_df_to_html(type_html = 3, list_df = list_df_flat, type_sector = 2, cnxn = engine)
    to_email = ['nngocphuonguyen@savills.com.vn']
    run_email(type_sector = 'FDI', email_type = 3, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

Missing values in dictionary
Sent email successfully


### TRACKING AUDIT

In [31]:
insert_to_tracking(list_df_flat, 'Tracking_FDI', engine)

Inserted audit tracking sucessfully
